### Introduction to the data

In [15]:
import sqlite3

conn = sqlite3.connect("nominations.db")
schema = conn.execute("pragma table_info(nominations);").fetchall()
for r in schema:
    print(r)

(0, 'false', 'INTEGER', 0, None, 0)
(1, 'Year', 'INTEGER', 0, None, 0)
(2, 'Category', 'TEXT', 0, None, 0)
(3, 'Nominee', 'TEXT', 0, None, 0)
(4, 'Won', 'INTEGER', 0, None, 0)
(5, 'Movie', 'TEXT', 0, None, 0)
(6, 'Character', 'TEXT', 0, None, 0)


In [17]:
first_ten = conn.execute("SELECT * FROM nominations limit 10;").fetchall()
for r in first_ten:
    print(r)

(0, 2010, 'Actor -- Leading Role', 'Javier Bardem', 0, 'Biutiful', 'Uxbal')
(1, 2010, 'Actor -- Leading Role', 'Jeff Bridges', 0, 'True Grit', 'Rooster Cogburn')
(2, 2010, 'Actor -- Leading Role', 'Jesse Eisenberg', 0, 'The Social Network', 'Mark Zuckerberg')
(3, 2010, 'Actor -- Leading Role', 'Colin Firth', 1, "The King's Speech", 'King George VI')
(4, 2010, 'Actor -- Leading Role', 'James Franco', 0, '127 Hours', 'Aron Ralston')
(5, 2010, 'Actor -- Supporting Role', 'Christian Bale', 1, 'The Fighter', 'Dicky Eklund')
(6, 2010, 'Actor -- Supporting Role', 'John Hawkes', 0, "Winter's Bone", 'Teardrop')
(7, 2010, 'Actor -- Supporting Role', 'Jeremy Renner', 0, 'The Town', 'James Coughlin')
(8, 2010, 'Actor -- Supporting Role', 'Mark Ruffalo', 0, 'The Kids Are All Right', 'Paul')
(9, 2010, 'Actor -- Supporting Role', 'Geoffrey Rush', 0, "The King's Speech", 'Lionel Logue')


### Execute Queries

In [22]:
conn.execute("CREATE TABLE ceremonies( \
             id integer, \
             Year integer, \
             Host text, \
             PRIMARY KEY (id));")

In [23]:
years_hosts = [(2010, "Steve Martin"),
               (2009, "Hugh Jackman"),
               (2008, "Jon Stewart"),
               (2007, "Ellen DeGeneres"),
               (2006, "Jon Stewart"),
               (2005, "Chris Rock"),
               (2004, "Billy Crystal"),
               (2003, "Steve Martin"),
               (2002, "Whoopi Goldberg"),
               (2001, "Steve Martin"),
               (2000, "Billy Crystal"),
            ]

In [24]:
insert_query = "INSERT INTO ceremonies (Year, Host) VALUES (?,?);"
conn.executemany(insert_query, years_hosts)

In [26]:
first_ten_c = conn.execute("SELECT * FROM ceremonies limit 10;").fetchall()
for r in first_ten_c:
    print(r)
    
schema_c = conn.execute("pragma table_info(ceremonies);").fetchall()
for r in schema_c:
    print(r)

(1, 2010, 'Steve Martin')
(2, 2009, 'Hugh Jackman')
(3, 2008, 'Jon Stewart')
(4, 2007, 'Ellen DeGeneres')
(5, 2006, 'Jon Stewart')
(6, 2005, 'Chris Rock')
(7, 2004, 'Billy Crystal')
(8, 2003, 'Steve Martin')
(9, 2002, 'Whoopi Goldberg')
(10, 2001, 'Steve Martin')
(0, 'id', 'integer', 0, None, 1)
(1, 'Year', 'integer', 0, None, 0)
(2, 'Host', 'text', 0, None, 0)


In [27]:
conn.execute("PRAGMA foreign_keys = ON;")

In [33]:
conn.execute("CREATE TABLE nominations_two( \
             id integer primary key, \
             category text, \
             nominee text, \
             movie text, \
             character text, \
             won integer, \
             ceremony_id integer, \
             FOREIGN KEY(ceremony_id) REFERENCES ceremonies(id));")

In [43]:
nom_query = '''
select ceremonies.id as ceremony_id, nominations.category as category, 
nominations.nominee as nominee, nominations.movie as movie, 
nominations.character as character, nominations.won as won
from nominations
inner join ceremonies 
on nominations.year == ceremonies.year
;
'''

joined_nominations = conn.execute(nom_query).fetchall()

In [46]:
print(conn.execute("select * from nominations_two limit 5;").fetchall())

[(1, 'Actor -- Leading Role', 'Javier Bardem', 'Biutiful', 'Uxbal', 0, None), (2, 'Actor -- Leading Role', 'Javier Bardem', 'Biutiful', 'Uxbal', 0, 1), (3, 'Actor -- Leading Role', 'Jeff Bridges', 'True Grit', 'Rooster Cogburn', 0, 1), (4, 'Actor -- Leading Role', 'Jesse Eisenberg', 'The Social Network', 'Mark Zuckerberg', 0, 1), (5, 'Actor -- Leading Role', 'Colin Firth', "The King's Speech", 'King George VI', 1, 1)]


In [45]:
insert_query = "INSERT INTO nominations_two (ceremony_id, category, nominee, movie, character, won) VALUES (?,?,?,?,?,?);"
conn.executemany(insert_query, joined_nominations)

In [47]:
conn.execute("DROP TABLE nominations;")
conn.execute("ALTER TABLE nominations_two RENAME TO nominations;")

In [50]:
conn.execute("drop table movies")
conn.execute("drop table actors")

create_movies = "create table movies (id integer primary key,movie text);"
create_actors = "create table actors (id integer primary key,actor text);"
create_movieactors = '''create table movies_actors (
id integer primary key,
movie_id integer,
actor_id integer,
FOREIGN KEY(movie_id) REFERENCES movies(id),
FOREIGN KEY(actor_id) REFERENCES actors(id));
'''

conn.execute(create_movies)
conn.execute(create_actors)
conn.execute(create_movieactors)


In [51]:
insert_movies = "insert into movies (movie) select distinct movie from nominations;"
insert_actors = "insert into actors (actor) select distinct nominee from nominations;"
conn.execute(insert_movies)
conn.execute(insert_actors)

print(conn.execute("select * from movies limit 5;").fetchall())
print(conn.execute("select * from actors limit 5;").fetchall())

[(1, 'Biutiful'), (2, 'True Grit'), (3, 'The Social Network'), (4, "The King's Speech"), (5, '127 Hours')]
[(1, 'Javier Bardem'), (2, 'Jeff Bridges'), (3, 'Jesse Eisenberg'), (4, 'Colin Firth'), (5, 'James Franco')]


In [52]:
pairs_query = "select movie,nominee from nominations;"
movie_actor_pairs = conn.execute(pairs_query).fetchall()
print(movie_actor_pairs)

[('Biutiful', 'Javier Bardem'), ('Biutiful', 'Javier Bardem'), ('True Grit', 'Jeff Bridges'), ('The Social Network', 'Jesse Eisenberg'), ("The King's Speech", 'Colin Firth'), ('127 Hours', 'James Franco'), ('The Fighter', 'Christian Bale'), ("Winter's Bone", 'John Hawkes'), ('The Town', 'Jeremy Renner'), ('The Kids Are All Right', 'Mark Ruffalo'), ("The King's Speech", 'Geoffrey Rush'), ('The Kids Are All Right', 'Annette Bening'), ('Rabbit Hole', 'Nicole Kidman'), ("Winter's Bone", 'Jennifer Lawrence'), ('Black Swan', 'Natalie Portman'), ('Blue Valentine', 'Michelle Williams'), ('The Fighter', 'Amy Adams'), ("The King's Speech", 'Helena Bonham Carter'), ('The Fighter', 'Melissa Leo'), ('True Grit', 'Hailee Steinfeld'), ('Animal Kingdom', 'Jacki Weaver'), ('Crazy Heart', 'Jeff Bridges'), ('Up in the Air', 'George Clooney'), ('A Single Man', 'Colin Firth'), ('Invictus', 'Morgan Freeman'), ('The Hurt Locker', 'Jeremy Renner'), ('Invictus', 'Matt Damon'), ('The Messenger', 'Woody Harrelso

In [53]:
join_table_insert = "insert into movies_actors (movie_id, actor_id) values ((select id from movies where movie == ?),(select id from actors where actor == ?));"
conn.executemany(join_table_insert,movie_actor_pairs)

print(conn.execute("select * from movies_actors limit 5;").fetchall())

[(1, 1, 1), (2, 1, 1), (3, 2, 2), (4, 3, 3), (5, 4, 4)]
